In [1]:
import time
import numpy as np
import pandas as pd

import multiprocessing as mp
from multiprocessing import Pool
from functools import partial

In [2]:
sale_path = 'sales_and_inventory_mentor_data\Sales_mentor\TT T03-2022_split_2.xlsx'
products_path = 'sales_and_inventory_mentor_data\MasterData\Productmaster.xlsx'
distribution_path ='InventoryAndSale_snapshot_data\MasterData\Distribution Channel.xlsx'

In [3]:
sale_df = pd.read_excel(sale_path)
sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87684 entries, 0 to 87683
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   month                      87684 non-null  int64 
 1   week                       87684 non-null  int64 
 2   site                       87684 non-null  int64 
 3   branch_id                  87684 non-null  int64 
 4   channel_id                 87684 non-null  object
 5   distribution_channel       87684 non-null  object
 6   distribution_channel_code  87684 non-null  object
 7   sold_quantity              87684 non-null  int64 
 8   cost_price                 87684 non-null  int64 
 9   net_price                  87684 non-null  int64 
 10  customer_id                87684 non-null  object
 11  product_id                 87684 non-null  object
dtypes: int64(7), object(5)
memory usage: 8.0+ MB


In [10]:
sale_df

,month,week,site,branch_id,channel_id,distribution_channel,distribution_channel_code,sold_quantity,cost_price,net_price,customer_id,product_id
0,2022003,202211,1200,1200,TGPP,Phát sinh,ZF2,1,203629,299455,c1cfc02d2,4576f65145124b6fb2884adfa197003dDEN37
1,2022003,202212,1150,1100,CHTT,Bán lẻ,FP,1,120174,177000,3389917a2,a42395a564364e5693b178dce514bd84NAU37
2,2022003,202211,1602,1600,CHTT,Bán lẻ,FP,1,226996,334000,9de04ba6f,07352d9ab2e4403b823219f660d2d1baXAM42
3,2022003,202212,1117,1100,CHTT,Bán lẻ,FP,1,173586,204000,ddb829703,43fcaa7920db44cd8f044fb58d1d27e0XAM42
4,2022003,202213,1600,1600,TGPP,Hợp đồng,ZF1,1,297098,333072,697997988,d4281d34e5674d74ae92ff4c38dfc27eXMN40
...,...,...,...,...,...,...,...,...,...,...,...,...
87679,2022003,202212,1114,1100,CHTT,Bán lẻ,FP,1,250364,368000,2ac4efa41,158124206d0a49aeb83cb13e19a91ea8NAU37
87680,2022003,202213,1120,1100,CHTT,Bán lẻ,FP,1,160232,236000,840519b59,9beccf0d1a174b2fbe42ac05d65fd7f2DEN42
87681,2022003,202212,1101,1100,CHTT,Bán lẻ,FP,1,50740,75000,dd5a95cfa,81bba395e1a3408693bd95d6148be3c2DEN38
87682,2022003,202211,1411,1400,CHTT,Bán lẻ,FP,1,223658,197000,6c3505931,22826feb50d84a2eb8c71cf03c7e8f05DEN38


In [ ]:
df = pd.read_excel(sale_path, dtype={'product_id':str})

In [4]:
product_df = pd.read_excel(products_path)
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94867 entries, 0 to 94866
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            94867 non-null  int64  
 1   index                 94867 non-null  int64  
 2   color                 94822 non-null  object 
 3   color_group           94867 non-null  object 
 4   listing_price         94840 non-null  float64
 5   price_group           94840 non-null  object 
 6   gender                94867 non-null  object 
 7   product_group         94867 non-null  object 
 8   detail_product_group  94867 non-null  object 
 9   shoe_product          94867 non-null  object 
 10  size_group            94867 non-null  object 
 11  size                  94722 non-null  float64
 12  age_group             94867 non-null  object 
 13  activity_group        94867 non-null  object 
 14  collection            94862 non-null  object 
 15  image_copyright    

In [5]:
distribution_df = pd.read_excel(distribution_path)
distribution_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3405 entries, 0 to 3404
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      3405 non-null   int64 
 1   index           3405 non-null   int64 
 2   site_store      3405 non-null   int64 
 3   b2b_b2c         3405 non-null   object
 4   channel_id      3405 non-null   object
 5   region          3405 non-null   object
 6   city_level      3405 non-null   object
 7   store_concept   3403 non-null   object
 8   trade_term      3405 non-null   object
 9   area_range      464 non-null    object
 10  store_type      3405 non-null   object
 11  urbanization    3405 non-null   object
 12  branch_area     3405 non-null   object
 13  address_2       3403 non-null   object
 14  address_3       3403 non-null   object
 15  showroom_area   470 non-null    object
 16  warehouse_area  456 non-null    object
 17  start_month     3376 non-null   object
 18  start_ye

In [6]:
feedback_matrix = pd.pivot_table(sale_df, values='sold_quantity', index='customer_id', columns='product_id', aggfunc='sum', fill_value=0)

In [7]:
feedback_matrix

product_id,000108ac94864037b89c6d7ce78d1134KEM35,000f226e34774f499b66d3c8d1704fe3DEN40,0010c5851d064b3e9692898cea014db1TRG37,00134b14581340878572570a534ec1a3DEN39,0013736f5aea4bd3a6cbf7320176d092DOO26,0018eddc01544dbc9857e8e26ea50b61XNH31,00192eb1861a40bfa0b0fb19bfa46c82XDG31,001e52fed7cb463ba533233c59c3ed51XNH28,002060dbf88f4ea6b5710b29a6b1c2b1DEN39,002095d0d1ec4b75a2956e5e2e1755a4XAM42,...,ffc243f613b747b1aeca30b6500e9a23XNH43,ffc60de6a3be46a7ab3373bd1bf1858bREU36,ffc81ee5f32b400e83963e2ec5c3c8c0XAM38,ffcf88d066134468939a286a47880bfbDEN38,ffd760d666bc4080925698e04d3ee0a2DEN37,ffda61a7d21f4c009283d2efd9952d8fDEN40,ffe65a593be142d294084661fc4cd7a1DEN35,fff0c748c1df4d3d87645319350a08c8XNH39,fff116f43dd8455e8ee2c5423c5db50eHOG28,fff4b6234a9248d6a65fbf6c31204cdaDEN29
customer_id,,,,,,,,,,,,,,,,,,,,,
004c2ae83,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,2
007e6bb72,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
008e9d4fd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00981a197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01347a866,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fc0dc8117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fc1944a2b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fc5887d8c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
ALL_USERS = distribution_df['customer_id'].unique().tolist()
ALL_ITEMS = product_df['product_id'].unique().tolist()